In [1]:
import torch
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingClassifier

data, y = torch.load('../input/titanic/data_v2.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

score_model = {}
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )
for i in range(100):
    print(f'epoch{i}')
    np.random.seed(i)

    std = StandardScaler()
    bagging_svm = BaggingClassifier(base_estimator=SVC(probability=True))
    bagging_svm.set_params(
        **{'n_estimators': 99, 'base_estimator__C': 36.06241178040971, 'base_estimator__gamma': 0.004735786412937079}
    )
    lr = LogisticRegression()
    adaboost = AdaBoostClassifier(base_estimator=SVC(probability=True))
    adaboost_tree = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
    adaboost_tree.set_params(**{'n_estimators': 349, 'learning_rate': 0.01, 'base_estimator__max_depth': 36,
                                'base_estimator__min_samples_split': 189, 'base_estimator__min_samples_leaf': 1,
                                'base_estimator__max_features': 25, 'base_estimator__max_leaf_nodes': 471}
                             )
    adaboost.set_params(
        **{'n_estimators': 65, 'learning_rate': 0.03694467169486954, 'base_estimator__C': 0.28335389417624707,
           'base_estimator__gamma': 0.007507640146291138}
    )
    gdbt = GradientBoostingClassifier()
    gdbt.set_params(
        **{'n_estimators': 300, 'learning_rate': 0.06504234821394586, 'max_depth': 3, 'min_samples_split': 2,
           'min_samples_leaf': 1, 'max_features': 14, 'max_leaf_nodes': 891}

    )
    knn = KNeighborsClassifier()
    knn.set_params(**{'n_neighbors': 6, 'p': 2, 'weights': 'uniform'}
                   )
    rf = RandomForestClassifier()
    rf.set_params(
        **{'n_estimators': 276, 'max_depth': 99, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 21,
           'max_leaf_nodes': 31}
    )

    svc = SVC(probability=True)
    svc.set_params(**{'C': 1.0121047403763643, 'gamma': 0.17953963922087096}
                   )
    tree = DecisionTreeClassifier()
    tree.set_params(
        **{'max_depth': 3, 'min_samples_split': 103, 'min_samples_leaf': 1, 'max_features': 24}

    )

    voting_model = VotingClassifier([('lr', lr), ('ada', adaboost), ('gdbt', gdbt),
                                     ('knn', knn), ('svc', svc), ('tree', tree),
                                     ('bagging_svm', bagging_svm), ], voting='soft', n_jobs=-1,
                                    verbose=2)
    pipe = Pipeline([
        ('std', std),
        ('voting', voting_model)
    ])

    pipe.fit(train_data, y)
    score = pipe.score(train_data, y)
    score_model[-score]=pipe
    print(f'epoch{i} {score}')

import pandas as pd
import heapq

keys=list(score_model.keys())
heapq.heapify(keys)
same_score=set()
for i in range(10):
    _ = heapq.heappop(keys)
    if _ in same_score:
        continue
    else:
        same_score.add(_)
    y_test = score_model[_].predict(test_data.values)
    result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
    result.to_csv(f'seed{i}.csv')

epoch0
epoch0 0.8540965207631874
epoch1
epoch1 0.8529741863075196
epoch2
epoch2 0.8540965207631874
epoch3
epoch3 0.8540965207631874
epoch4
epoch4 0.8540965207631874
epoch5
epoch5 0.8540965207631874
epoch6
epoch6 0.8540965207631874
epoch7
epoch7 0.8552188552188552
epoch8
epoch8 0.8529741863075196
epoch9
epoch9 0.8552188552188552
epoch10
epoch10 0.8552188552188552
epoch11
epoch11 0.8540965207631874
epoch12
epoch12 0.8540965207631874
epoch13
epoch13 0.8552188552188552
epoch14
epoch14 0.8540965207631874
epoch15
epoch15 0.856341189674523
epoch16
epoch16 0.8540965207631874
epoch17
epoch17 0.8529741863075196
epoch18
epoch18 0.8529741863075196
epoch19
epoch19 0.8540965207631874
epoch20
epoch20 0.8552188552188552
epoch21
epoch21 0.8540965207631874
epoch22
epoch22 0.8552188552188552
epoch23
epoch23 0.8540965207631874
epoch24
epoch24 0.8540965207631874
epoch25
epoch25 0.8540965207631874
epoch26
epoch26 0.8552188552188552
epoch27
epoch27 0.8540965207631874
epoch28
epoch28 0.8552188552188552
epoch2

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid 

IndexError: index out of range